# NYC Mamdani Millionaire Income Tax Analysis (2026)

This notebook analyzes the fiscal and distributional impacts of Mayor-elect Zohran Mamdani's proposed income tax for New York City.

## Baseline (Current Law)
- Current NYC income tax structure

## Reform
- Mamdani Millionaire Income Tax proposal

## Metrics
We calculate:
- Budgetary impact (revenue raised)
- Number and percentage of people/households affected
- Average change in net income for those affected
- Distributional analysis by income decile

In [9]:
from policyengine_us import Microsimulation
from policyengine_us.reforms.local.ny.mamdani_income_tax import nyc_mamdani_income_tax
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

NYC_DATASET = "hf://policyengine/policyengine-us-data/cities/NYC.h5"
YEAR = 2026

# Create combined reform: structural reform + enable the parameter
param_reform = Reform.from_dict(
    {
        "gov.local.ny.mamdani_income_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

# Combine reforms: parameter reform first, then structural reform
mamdani_reform = (param_reform, nyc_mamdani_income_tax)

## Helper Functions

In [10]:
def calculate_affected(baseline_sim, reform_sim, period=YEAR):
    """
    Calculate people affected by the reform (losers who pay more taxes).
    Returns weighted counts, percentages, and average changes.
    """
    # Get household-level income change
    baseline_income = np.array(baseline_sim.calculate("household_net_income", period=period, map_to="household"))
    reform_income = np.array(reform_sim.calculate("household_net_income", period=period, map_to="household"))
    household_weight = np.array(baseline_sim.calculate("household_weight", period=period))
    income_change = reform_income - baseline_income
    
    # Get person-level data
    household_id_person = np.array(baseline_sim.calculate("household_id", period=period, map_to="person"))
    household_id_household = np.array(baseline_sim.calculate("household_id", period=period, map_to="household"))
    person_weight = np.array(baseline_sim.calculate("person_weight", period=period))
    
    # Create mapping of household_id to income_change
    income_change_dict = dict(zip(household_id_household, income_change))
    
    # Map income change to each person
    person_income_change = np.array([income_change_dict.get(hh_id, 0) for hh_id in household_id_person])
    
    # Weighted count of people who are losers (lost more than $1 - paying more taxes)
    losers_mask = person_income_change < -1
    people_losing = person_weight[losers_mask].sum()
    
    total_people = person_weight.sum()
    
    # Calculate percentage
    pct_losers = (people_losing / total_people * 100) if total_people > 0 else 0
    
    # Households affected
    losing_hh_mask = income_change < -1
    households_losing = household_weight[losing_hh_mask].sum()
    total_households = household_weight.sum()
    pct_households_losing = (households_losing / total_households * 100) if total_households > 0 else 0
    
    # Average loss for affected households (weighted)
    avg_loss = np.average(income_change[losing_hh_mask], weights=household_weight[losing_hh_mask]) if losing_hh_mask.sum() > 0 else 0
    
    return {
        "people_losing": people_losing,
        "total_people": total_people,
        "pct_losers": pct_losers,
        "households_losing": households_losing,
        "total_households": total_households,
        "pct_households_losing": pct_households_losing,
        "avg_loss": avg_loss
    }

def calculate_decile_impacts(baseline_sim, reform_sim, period=YEAR):
    """
    Calculate average income change by income decile.
    """
    from microdf import MicroSeries
    
    baseline_net_income = baseline_sim.calculate("household_net_income", map_to="household", period=period)
    reform_net_income = reform_sim.calculate("household_net_income", map_to="household", period=period)
    
    count_people = baseline_sim.calculate("household_count_people", period=period)
    household_weight = baseline_sim.calculate("household_weight", period=period)
    
    weighted_income = MicroSeries(
        baseline_net_income, weights=household_weight * count_people
    )
    decile = weighted_income.decile_rank().values
    
    household_income_decile = (np.where(baseline_net_income < 0, -1, decile)).astype(int)
    
    income_change = reform_net_income - baseline_net_income
    
    # Calculate average change by decile
    average_change = income_change.groupby(household_income_decile).mean()
    
    # Filter to valid deciles (1-10)
    average_change = average_change[average_change.index > 0]
    
    return average_change

def format_currency(value):
    """Format value as currency in millions or billions."""
    if abs(value) >= 1e9:
        return f"${value/1e9:.2f}B"
    else:
        return f"${value/1e6:.2f}M"

## Load Simulations

In [11]:
print("Loading baseline (current NYC income tax structure)...")
baseline = Microsimulation(dataset=NYC_DATASET)
print("Baseline loaded")

print("\nLoading reform (Mamdani Millionaire Income Tax)...")
reform_sim = Microsimulation(dataset=NYC_DATASET, reform=mamdani_reform)
print("Reform loaded")

print("\n" + "="*60)
print("All simulations ready!")
print("="*60)

Loading baseline (current NYC income tax structure)...
Baseline loaded

Loading reform (Mamdani Millionaire Income Tax)...
Reform loaded

All simulations ready!


## Calculate Impacts

In [12]:
# Revenue impact - calculated as decrease in household net income (revenue to city)
baseline_hh_income = baseline.calculate("household_net_income", period=YEAR, map_to="household").sum()
reform_hh_income = reform_sim.calculate("household_net_income", period=YEAR, map_to="household").sum()
revenue_raised = baseline_hh_income - reform_hh_income  # Positive if reform raises revenue

# Affected population
affected = calculate_affected(baseline, reform_sim)

# Decile impacts
decile_impacts = calculate_decile_impacts(baseline, reform_sim)

print("All impacts calculated")

All impacts calculated


## Results Summary

In [13]:
print("\n" + "="*80)
print(f"NYC MAMDANI MILLIONAIRE INCOME TAX IMPACTS ({YEAR})")
print("Baseline: Current NYC income tax | Reform: Mamdani Millionaire Income Tax")
print("="*80)

print(f"\n{'BUDGETARY IMPACT':=^80}")
print(f"Revenue raised:                {format_currency(revenue_raised)}")

print(f"\n{'AFFECTED POPULATION':=^80}")
print(f"People paying more taxes:      {affected['people_losing']:,.0f} ({affected['pct_losers']:.2f}% of population)")
print(f"Households paying more taxes:  {affected['households_losing']:,.0f} ({affected['pct_households_losing']:.2f}% of households)")
print(f"Average change for affected:   ${affected['avg_loss']:,.2f}")
print("="*80)


NYC MAMDANI MILLIONAIRE INCOME TAX IMPACTS (2026)
Baseline: Current NYC income tax | Reform: Mamdani Millionaire Income Tax

================================BUDGETARY IMPACT================================
Revenue raised:                $10.13B

==============================AFFECTED POPULATION===============================
People paying more taxes:      628,033 (9.11% of population)
Households paying more taxes:  184,633 (7.84% of households)
Average change for affected:   $-54,859.30


## Distributional Analysis by Income Decile

In [14]:
print("\n" + "="*80)
print("AVERAGE HOUSEHOLD INCOME CHANGE BY DECILE")
print("="*80)
print(f"{'Decile':<10} {'Average Change':>20}")
print("-"*30)
for decile in range(1, 11):
    if decile in decile_impacts.index:
        change = decile_impacts[decile]
        print(f"{decile:<10} ${change:>18,.2f}")
print("="*80)
print("Note: Negative values indicate income loss (higher taxes)")


AVERAGE HOUSEHOLD INCOME CHANGE BY DECILE
Decile           Average Change
------------------------------
1          $              0.00
2          $              0.00
3          $              0.00
4          $              0.00
5          $              0.00
6          $              0.00
7          $              0.00
8          $              0.00
9          $             -6.73
10         $        -51,204.51
Note: Negative values indicate income loss (higher taxes)


## Export Results

In [15]:
# Create results DataFrame
results = [
    {
        "Scenario": "Mamdani Income Tax",
        "Description": "Millionaire Income Tax for NYC",
        "Year": YEAR,
        "Revenue Raised": format_currency(revenue_raised),
        "% Population Affected": f"{affected['pct_losers']:.2f}%",
        "% Households Affected": f"{affected['pct_households_losing']:.2f}%",
        "Avg Change for Affected": f"${affected['avg_loss']:,.2f}"
    }
]

df_results = pd.DataFrame(results)

print("\n" + "="*110)
print("NYC MAMDANI INCOME TAX REFORM SUMMARY")
print("="*110)
print(df_results.to_string(index=False))
print("="*110)

# Export to CSV
df_results.to_csv("nyc_mamdani_income_tax_results.csv", index=False)
print("\nExported to: nyc_mamdani_income_tax_results.csv")


NYC MAMDANI INCOME TAX REFORM SUMMARY
          Scenario                    Description  Year Revenue Raised % Population Affected % Households Affected Avg Change for Affected
Mamdani Income Tax Millionaire Income Tax for NYC  2026        $10.13B                 9.11%                 7.84%             $-54,859.30

Exported to: nyc_mamdani_income_tax_results.csv


In [16]:
# Export decile impacts
decile_df = pd.DataFrame({
    'Decile': decile_impacts.index,
    'Average Income Change': decile_impacts.values
})
decile_df.to_csv("nyc_mamdani_decile_impacts.csv", index=False)
print("Decile impacts exported to: nyc_mamdani_decile_impacts.csv")

Decile impacts exported to: nyc_mamdani_decile_impacts.csv
